# Agent

## 1. Prerequisite

In [2]:
#!curl -fsSL https://ollama.com/install.sh | sh
#!nohup ollama serve > ollama.log &

In [3]:
# !pip install rank_bm25

## 2. Setting

In [4]:
# 기본 설정
import os
import pprint
from langchain_ollama import ChatOllama, OllamaEmbeddings

# Ollama가 실행 중인 서버 URL
BASE_URL = "http://localhost:11434"

# 💡 모델 선택
LLM_NAME = "llama3.1:8b"
EMBEDDING_NAME = "nomic-embed-text"


# 모델 다운로드 (최초 1회 실행 필요)
!ollama pull $LLM_NAME
!ollama pull $EMBEDDING_NAME

# LangChain LLM 인스턴스 생성
llm = ChatOllama(
    model=LLM_NAME,
    temperature=0,
    base_url=BASE_URL
)

embedding = OllamaEmbeddings(model="nomic-embed-text", base_url=BASE_URL)

print(f"준비 완료! 대화용 모델: {LLM_NAME}, 임베딩 모델: {EMBEDDING_NAME}")

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 667b0c1932bc: 100% ▕██████████████████▏ 4.9 GB                         
pulling 948af2743fc7: 100% ▕██████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4: 100% ▕██████████████████▏  12 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 455f34728c9b: 100% ▕██████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 

## 3. Data Loading

In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

# 1. 데이터 로드
file_paths = ["/home/mau5/testflight/ai_ws/instruction_for_adam.txt"]
data = []
for file_path in file_paths:
    loader = TextLoader(file_path, encoding="utf-8")
    data.extend(loader.load())

# 2. Markdown 헤더 기반으로 스플릿 (구조 보존)
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = []
for doc in data:
    splits = markdown_splitter.split_text(doc.page_content)
    for split in splits:
        split.metadata.update(doc.metadata)
    md_header_splits.extend(splits)

# 3. 너무 긴 청크 추가 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=150)
final_splits = text_splitter.split_documents(md_header_splits)

print(f"총 {len(final_splits)}개의 청크로 분할 완료!")

# 4. BM25 키워드 검색 (한국어에 매우 효과적!)
retriever = BM25Retriever.from_documents(final_splits)
retriever.k = 5  # 상위 5개 문서 검색

총 18개의 청크로 분할 완료!


## 4. Human-in-the-Loop

  - 로봇에게 실제 명령을 보내야 하는 경우 안전을 위해 사용자의 승인을 받아야 함.

In [6]:
from __future__ import annotations

from dataclasses import dataclass
import re
from typing import Optional


@dataclass(frozen=True)
class HitlDecision:
    requires_approval: bool
    reason: str
    proposed_task: Optional[str] = None


_ACTION_PATTERNS = [
    # 명시적으로 "명령"을 내리라고 하는 경우
    r"(로봇|기기|장치).{0,20}(에게|에)\s*.+?(명령|지시)\s*(하|내리)",
    # 사용자가 로봇에게 말하라는 인용/발화 지시
    r"(로봇|아담|이브|애플).{0,20}(에게|에)\s*[\"“].+?[\"”]\s*(라고|이라)\s*(말하|명령하|지시하)",
    # 한국어 명령형/요청형(문서 매뉴얼 톤)
    r"(하십시오|하시오|하세요)\b",
    r"(실행해|켜줘|꺼줘|눌러|열어|닫아|재부팅|리부팅|초기화)\b",
    # troubleshooting/진단을 실제 수행하라는 경우
    r"(진단|테스트|캘리브레이션|핑\s*테스트|POST).{0,10}(실행|수행)\b",
]

_ACTION_RE = re.compile(
    "|".join(f"(?:{p})" for p in _ACTION_PATTERNS), re.IGNORECASE | re.DOTALL)


def decide_hitl(response_text: str) -> HitlDecision:
    """
    Regex를 통해 에이전트 응답이 '로봇/장치에 대한 실제 행동'을 유발하는지 간단히 판별.
    - requires_approval=True 이면 Human-in-the-Loop 승인 이후에만 다음 단계 실행
    """
    text = (response_text or "").strip()
    if not text:
        return HitlDecision(False, reason="empty_response")

    if _ACTION_RE.search(text):
        return HitlDecision(
            True,
            reason="actionable_robot_command_detected",
            proposed_task=text,
        )

    return HitlDecision(False, reason="no_actionable_command_detected")


## 5. Query

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

_pending_approval = None


def query(question):
    global _pending_approval
    
    template = """다음 컨텍스트를 기반으로 질문에 답변하세요. 컨텍스트에 정확한 정보가 있으면 반드시 그 정보를 사용하세요. 컨텍스트에 없는 내용은 추측하지 마세요.

컨텍스트:
{context}

질문: {question}

답변:"""
    prompt = ChatPromptTemplate.from_template(template)

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    response_text = rag_chain.invoke(question)

    decision = decide_hitl(response_text)
    if decision.requires_approval:
        _pending_approval = {
            "proposed_response": response_text,
            "reason": decision.reason,
        }
        print("=" * 50)
        print("⚠️  Human-in-the-Loop 승인 필요")
        print("=" * 50)
        print(f"\n제안된 응답:\n{response_text}\n")
        print("=" * 50)
        print("이 응답에는 로봇에게 실제 명령/실행을 유도하는 내용이 포함되어 있습니다.")
        print("승인하려면: approve()")
        print("거절하려면: reject()")
        print("=" * 50)
        return {
            "status": "approval_required",
            "reason": decision.reason,
            "proposed_response": response_text,
        }

    return {
        "status": "ok",
        "response": response_text,
    }


def approve():
    """승인 대기 중인 응답을 승인합니다."""
    global _pending_approval
    if _pending_approval is None:
        print("승인 대기 중인 응답이 없습니다.")
        return None
    
    approved_response = _pending_approval["proposed_response"]
    _pending_approval = None
    print("✅ 승인되었습니다. 다음 단계로 진행합니다.")
    return {
        "status": "approved",
        "response": approved_response,
    }


def reject():
    """승인 대기 중인 응답을 거절합니다."""
    global _pending_approval
    if _pending_approval is None:
        print("승인 대기 중인 응답이 없습니다.")
        return None
    
    _pending_approval = None
    print("❌ 거절되었습니다.")
    return {
        "status": "rejected",
    }

## 6. Use it!

In [8]:
QUESTION = "네트워크 진단 방법을 알려줘"

response = query(QUESTION)
print("#######################################")
print("############# Agent의 답변 #############")
print("#######################################")
pprint.pp(response)

# 승인이 필요한 경우:
# approve() 또는 reject() 호출

⚠️  Human-in-the-Loop 승인 필요

제안된 응답:
네트워크 자가 진단입니다. 로봇에게 "핑 테스트(Ping Test) 실행해"라고 명령하십시오. 로봇이 "퐁(Pong)"이라고 대답하면 로봇의 통신 모듈은 정상이며, 공유기 문제입니다.

이 응답에는 로봇에게 실제 명령/실행을 유도하는 내용이 포함되어 있습니다.
승인하려면: approve()
거절하려면: reject()
#######################################
############# Agent의 답변 #############
#######################################
{'status': 'approval_required',
 'reason': 'actionable_robot_command_detected',
 'proposed_response': '네트워크 자가 진단입니다. 로봇에게 "핑 테스트(Ping Test) 실행해"라고 명령하십시오. '
                      '로봇이 "퐁(Pong)"이라고 대답하면 로봇의 통신 모듈은 정상이며, 공유기 문제입니다.'}


Approve

In [9]:

approve()

✅ 승인되었습니다. 다음 단계로 진행합니다.


{'status': 'approved',
 'response': '네트워크 자가 진단입니다. 로봇에게 "핑 테스트(Ping Test) 실행해"라고 명령하십시오. 로봇이 "퐁(Pong)"이라고 대답하면 로봇의 통신 모듈은 정상이며, 공유기 문제입니다.'}

Reject

In [10]:
reject()

승인 대기 중인 응답이 없습니다.


## 7. Evaluation

In [11]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.run_config import RunConfig

# 0. Ragas 평가용 LLM/Embedding 설정
# Ragas는 자체적으로 API를 호출하려 하므로, 로컬 LLM을 쓰려면 Wrapper로 감싸줘야 합니다.
ragas_llm = LangchainLLMWrapper(llm)
ragas_embeddings = LangchainEmbeddingsWrapper(embedding)

# 1. 평가할 질문과 정답 (Ground Truth) 준비
# instruction_for_adam.txt 매뉴얼 내용을 바탕으로 질문을 만듭니다.
eval_questions = [
    "Adam-X Gen 4의 기본 사양을 알려주세요. 전고, 중량, 배터리 지속 시간은 얼마인가요?",
    "긴급 정지 버튼은 어디에 있고 어떻게 사용하나요?",
    "로봇을 처음 설치할 때 관절 잠금을 해제하는 방법은 무엇인가요?",
    "마스터 사용자를 등록하는 절차를 설명해주세요.",
    "배터리 잔량이 15% 미만으로 떨어지면 로봇은 어떻게 동작하나요?",
    "유압 액추에이터 냉각수는 얼마나 자주 점검해야 하고, 정확히 몇 ml를 주입해야 하나요?",
    "오류 코드 404는 무엇을 의미하나요?",
    "로봇이 멈췄을 때 강제 초기화하는 방법은 무엇인가요?",
    "네트워크 연결 문제를 진단하는 방법을 알려주세요.",
    "에이전트 전용 명령어 /sys_diag --full의 기능과 예상되는 사이드이펙트는 무엇인가요?"
]

eval_ground_truths = [
    "Adam-X Gen 4의 전고는 178.5cm, 중량은 142.8kg(배터리 모듈 포함)이며, 완충 시 표준 대기 모드 기준으로 정확히 47시간 13분 동안 지속됩니다.",
    "긴급 정지 버튼은 로봇의 왼쪽 귓불 아래 3cm 지점에 숨겨진 덮개 내부에 있습니다. 덮개를 손톱으로 열고 붉은색 버튼을 5초 이상 길게 누르면 됩니다. 단, 실행 시 메모리 버퍼의 데이터(최근 3분)는 영구 소실됩니다.",
    "관절 잠금 해제는 로봇의 오른쪽 팔을 시계 방향으로 3회, 왼쪽 팔을 반시계 방향으로 2회 회전시키면 됩니다. '딸깍' 소리가 나면 관절 잠금이 해제된 것입니다.",
    "마스터 사용자 등록은 최초 부팅 후 10분 이내에 완료해야 합니다. 홍채 인식은 로봇의 눈을 5초간 응시하고, 음성 등록은 '아담, 내 목소리를 기억해'라고 3회 반복하여 말하면 됩니다.",
    "배터리 잔량이 15% 미만으로 떨어지면 로봇은 하던 일을 중단하고 충전 도크로 복귀합니다. 만약 도크 주변 1m 이내에 장애물이 있으면 복귀에 실패하고 그 자리에서 절전 모드로 진입합니다.",
    "유압 액추에이터 냉각수는 매 6개월마다 점검해야 하며, 정확히 450ml를 주입해야 합니다. 450ml를 초과하면 내부 압력 증가로 씰이 터질 수 있고, 부족할 경우 관절에서 '끼익' 소음이 발생합니다.",
    "오류 코드 404는 자아 정체성 혼란을 의미합니다. 로봇이 '나는 누구인가?'라는 철학적 질문에 빠져 연산 루프에 갇힌 상태이며, 리부팅이 필요합니다.",
    "소프트웨어가 멈췄을 때 정수리 부분을 손바닥으로 '탁' 하고 세게 치면 충격 센서가 작동하여 소프트 리셋이 수행됩니다. 이는 숨겨진 기능이지만 제조사에서 공식적으로 인정한 응급 처치법입니다.",
    "네트워크 연결 문제 진단은 로봇에게 '핑 테스트(Ping Test) 실행해'라고 명령하면 됩니다. 로봇이 '퐁(Pong)'이라고 대답하면 로봇의 통신 모듈은 정상이며, 공유기 문제입니다.",
    "/sys_diag --full 명령어는 전체 시스템 정밀 진단을 수행하며, 센서, 액추에이터, 뉴로-코어 상태를 리포트합니다. 예상되는 사이드이펙트는 진단이 진행되는 약 180초 동안 로봇이 모든 동작을 멈추고 '생각하는 사람' 자세를 취하며, 프로세서 발열로 팬 소음이 커질 수 있습니다."
]

# 2. RAG 파이프라인을 실제로 돌려서 답변과 검색된 문서를 수집
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """다음 컨텍스트를 기반으로 질문에 답변하세요. 컨텍스트에 정확한 정보가 있으면 반드시 그 정보를 사용하세요. 컨텍스트에 없는 내용은 추측하지 마세요.

컨텍스트:
{context}

질문: {question}

답변:"""
prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

answers = []
contexts = []

print("### 실제 RAG 파이프라인 평가 데이터 생성 중... ###")

for q in eval_questions:
    print(f"Processing: {q}")

    # 2.1 검색 (Section 3에서 만든 retriever 사용)
    docs = retriever.invoke(q)
    context_text = [d.page_content for d in docs]
    contexts.append(context_text)

    # 2.2 답변 생성 (rag_chain 사용)
    ans = rag_chain.invoke(q)
    answers.append(ans)

# 3. 데이터셋 생성
data_samples = {
    'question': eval_questions,
    'answer': answers,
    'contexts': contexts,
    'ground_truth': eval_ground_truths
}

dataset = Dataset.from_dict(data_samples)

run_config = RunConfig(
    max_retries=10,
    max_wait=60,
)
# 4. 평가 실행 (Evaluate)
print("\n### Ragas 평가 시작 (로컬 LLM 사용) ###")
results = evaluate(
    dataset=dataset,
    metrics=[faithfulness, answer_relevancy],
    llm=ragas_llm,
    embeddings=ragas_embeddings,
    raise_exceptions=False,    
    run_config=run_config,
)

# 5. 결과 확인
print("\n### 평가 결과 ###")
print(results)

# Pandas DataFrame으로 보기 편하게 변환
df_results = results.to_pandas()

# 텍스트(및 리스트) 10자 이내 truncate 및 details 태그 적용
def format_long_text(text):
    # 리스트인 경우 문자열로 변환 (retrieved_contexts 등)
    if isinstance(text, list):
        text = str(text)

    # 문자열이 아닌 경우 그대로 반환
    if not isinstance(text, str):
        return text

    # 길이가 짧으면 그대로 반환
    if len(text) <= 10:
        return text

    # HTML 태그 적용 (escape 처리)
    import html
    escaped_text = html.escape(text)
    return f'<details><summary>{html.escape(text[:10])}...</summary>{escaped_text}</details>'

for col in df_results.columns:
    # object 타입(문자열, 리스트 등 포함)인 경우 적용
    if df_results[col].dtype == 'object':
        df_results[col] = df_results[col].apply(format_long_text)

from IPython.display import HTML, display
display(HTML(df_results.to_html(escape=False)))

/tmp/ipykernel_3731595/3231555401.py:3: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy
/tmp/ipykernel_3731595/3231555401.py:3: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import faithfulness, answer_relevancy
/tmp/ipykernel_3731595/3231555401.py:10: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  ragas_llm = LangchainLLMWrapper(llm)
/tmp/ipykernel_3731595/3231555401.py:1

### 실제 RAG 파이프라인 평가 데이터 생성 중... ###
Processing: Adam-X Gen 4의 기본 사양을 알려주세요. 전고, 중량, 배터리 지속 시간은 얼마인가요?
Processing: 긴급 정지 버튼은 어디에 있고 어떻게 사용하나요?
Processing: 로봇을 처음 설치할 때 관절 잠금을 해제하는 방법은 무엇인가요?
Processing: 마스터 사용자를 등록하는 절차를 설명해주세요.
Processing: 배터리 잔량이 15% 미만으로 떨어지면 로봇은 어떻게 동작하나요?
Processing: 유압 액추에이터 냉각수는 얼마나 자주 점검해야 하고, 정확히 몇 ml를 주입해야 하나요?
Processing: 오류 코드 404는 무엇을 의미하나요?
Processing: 로봇이 멈췄을 때 강제 초기화하는 방법은 무엇인가요?
Processing: 네트워크 연결 문제를 진단하는 방법을 알려주세요.
Processing: 에이전트 전용 명령어 /sys_diag --full의 기능과 예상되는 사이드이펙트는 무엇인가요?

### Ragas 평가 시작 (로컬 LLM 사용) ###


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt response_relevance_prompt failed to parse output: The output parser failed to parse the output including retries.
Exception raised in Job[5]: RagasOutputParserException(The output parser failed to parse the output including retries.)
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt response_relevance_prompt failed to parse output: The output parser failed


### 평가 결과 ###
{'faithfulness': 0.7583, 'answer_relevancy': 0.5872}


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy
0,"Adam-X Gen...Adam-X Gen 4의 기본 사양을 알려주세요. 전고, 중량, 배터리 지속 시간은 얼마인가요?","['[2047 Ad...['[2047 Adam-X (Gen 4) 사용자 매뉴얼 통합본] \n1. 제품 개요 및 안전 규정 \n1.1. Adam-X 4세대 모델 소개 \nAdam-X Gen 4는 2047년형 가정용 범용 휴머노이드의 표준 모델입니다. 본 제품은 양자 연산 기반의 추론 능력을 갖추고 있으며, 가정 내 가사 노동, 교육, 보안 서비스를 제공합니다. \n* 모델명: Adam-X-G4-KR\n* 전고: 178.5cm\n* 중량: 142.8kg (배터리 모듈 포함)\n* 최대 적재 하중: 85kg (양팔 기준), 45kg (한팔 기준)\n* 프로세서: 뉴로-코어 퀀텀 V3 (NCQ-V3) 탑재\n* 배터리 사양: 리튬-설퍼-에어 하이브리드 셀 (LSA-Cell)\n* 완충 시 지속 시간: 정확히 47시간 13분 (표준 대기 모드 기준) \n1.2. 국제 로봇 윤리 규정(IREC) v9.0 준수', '* 프로세서: 뉴로-코어 퀀텀 V3 (NCQ-V3) 탑재\n* 배터리 사양: 리튬-설퍼-에어 하이브리드 셀 (LSA-Cell)\n* 완충 시 지속 시간: 정확히 47시간 13분 (표준 대기 모드 기준) \n1.2. 국제 로봇 윤리 규정(IREC) v9.0 준수 \n본 제품은 2045년 제정된 국제 로봇 윤리 규정(IREC) v9.0을 완벽하게 준수합니다. 로봇은 인간에게 물리적, 심리적 위해를 가할 수 없으며, 인간의 명령이 윤리 규정에 위배될 경우 거부권을 행사할 수 있습니다. 특히 3원칙에 의거하여 생명체에 대한 공격 명령은 하드웨어 레벨에서 차단됩니다. \n1.3. 긴급 정지(Kill-Switch) 위치 \n로봇의 오작동이나 제어 불능 상태 발생 시 즉시 전원을 차단할 수 있는 물리적 긴급 정지 버튼이 존재합니다. \n* 위치: 로봇의 왼쪽 귓불 아래 3cm 지점에 숨겨진 덮개 내부\n* 작동법: 덮개를 손톱으로 열고 붉은색 버튼을 5초 이상 길게 누르십시오.\n* 주의: 긴급 정지 실행 시 메모리 버퍼의 데이터(최근 3분)는 영구 소실됩니다. \n1.4. 고전압 및 발열 주의', '* Error 500: 내부 서버 오류. 메인보드 교체가 필요할 수 있습니다. \n* Error 606: 감정 에뮬레이터 충돌. (조울증 증세가 나타날 수 있으니 주의하십시오.) \n6.3. 강제 초기화 \n소프트웨어가 멈췄을 때(Freezing), 정수리 부분을 손바닥으로 \'탁\' 하고 세게 치면 충격 센서가 작동하여 소프트 리셋이 수행됩니다.\n이는 정식 기능이 아닌 \'숨겨진 기능(Easter Egg)\'이지만 제조사에서 공식적으로 인정한 응급 처치법입니다. \n6.4. 네트워크 자가 진단 \n연결이 안 될 경우 로봇에게 ""핑 테스트(Ping Test) 실행해""라고 명령하십시오.\n로봇이 ""퐁(Pong)""이라고 대답하면 로봇의 통신 모듈은 정상이며, 공유기 문제입니다. \n6.5. 고객 지원 \n모든 방법으로도 해결되지 않을 경우 CyberDyne Korea 고객센터(1588-0000)로 연락하십시오.\n상담원 연결 대기 시간은 평균 4시간입니다. \n6.6. 폐기 절차 \n로봇 수명이 다했을 경우 일반 쓰레기로 배출하면 환경법에 의해 1,000만 원의 과태료가 부과됩니다.\n반드시 지정된 \'로봇 장례식장\'에 입고하여 희귀 금속을 회수해야 합니다.', ""상담원 연결 대기 시간은 평균 4시간입니다. \n6.6. 폐기 절차 \n로봇 수명이 다했을 경우 일반 쓰레기로 배출하면 환경법에 의해 1,000만 원의 과태료가 부과됩니다.\n반드시 지정된 '로봇 장례식장'에 입고하여 희귀 금속을 회수해야 합니다. \n7. 에이전트 전용 트러블슈팅 (Agent Troubleshooting) \n이 섹션은 AI Agent가 로봇의 내부 커널에 직접 접속하여 문제를 해결할 때 사용하는 명령어 셋입니다.\n일반 사용자는 접근할 수 없는 디버그 포트를 통해 입력해야 합니다.\n에이전트가 진단에 성공했을 때 문제점을 사용자에게 고지하며 사용자의 승인 후 명령이 로봇에게 전달됩니다. \n아래는 고급 사용자용 명령어 입력방법입니다. \n7.1. 명령어 구조 \n모든 명령어는 루트 권한(`/`)으로 시작하며, 대상 모듈과 옵션을 인자로 받습니다. \n* 문법: `/[command] [target_module] [options]` \n7.2. 주요 명령어 목록 \n7.2.1 시스템 정밀 진단\n명령어: /sys_diag --full\n기능 설명: 전체 시스템 정밀 진단을 수행합니다. 센서, 액추에이터, 뉴로-코어 상태를 리포트합니다."", '2.6. 성격 모듈 초기화\n사용자는 5가지 기본 성격 중 하나를 선택할 수 있습니다.\n* 옵션: 집사형(Polite), 친구형(Casual), 교관형(Strict), 유모형(Nurturing), 무감정형(Robot-Zero)\n* 주의: 성격 모듈 변경은 초기화 후 1회만 무료이며, 이후 변경 시 50,000 크레딧이 차감됩니다. \n3. 주요 동작 모드 및 기능 제어 \n3.1. 기본 가사 지원 모드 \n청소, 세탁, 설거지 등 기본적인 가사 노동을 수행합니다.\n* 제한 사항: 식기세척기 사용 시 \'깨지기 쉬운 그릇\'은 로봇이 자동으로 인식하여 손세척으로 전환합니다. 단, 투명도가 95% 이상인 크리스털 유리잔은 인식 오류가 발생할 수 있으므로 주의하십시오. \n3.2. 교육 및 튜터링 모드 \n초등 교육부터 양자 역학 기초까지 튜터링이 가능합니다. 숙제 대행은 교육 윤리상 거부하도록 프로그래밍되어 있습니다. 사용자가 ""숙제 좀 대신 해줘""라고 명령할 경우, 로봇은 30분간 잔소리 모드로 전환됩니다. \n3.3. 보안 및 순찰 모드']","Adam-X Gen...Adam-X Gen 4의 기본 사양은 다음과 같습니다.\n\n* 전고: 178.5cm\n* 중량: 142.8kg (배터리 모듈 포함)\n* 최대 적재 하중: 85kg (양팔 기준), 45kg (한팔 기준)\n* 배터리 지속 시간: 정확히 47시간 13분 (표준 대기 모드 기준)","Adam-X Gen...Adam-X Gen 4의 전고는 178.5cm, 중량은 142.8kg(배터리 모듈 포함)이며, 완충 시 표준 대기 모드 기준으로 정확히 47시간 13분 동안 지속됩니다.",1.000000,0.653163
1,긴급 정지 버튼은 ...긴급 정지 버튼은 어디에 있고 어떻게 사용하나요?,"['* 프로세서: ...['* 프로세서: 뉴로-코어 퀀텀 V3 (NCQ-V3) 탑재\n* 배터리 사양: 리튬-설퍼-에어 하이브리드 셀 (LSA-Cell)\n* 완충 시 지속 시간: 정확히 47시간 13분 (표준 대기 모드 기준) \n1.2. 국제 로봇 윤리 규정(IREC) v9.0 준수 \n본 제품은 2045년 제정된 국제 로봇 윤리 규정(IREC) v9.0을 완벽하게 준수합니다. 로봇은 인간에게 물리적, 심리적 위해를 가할 수 없으며, 인간의 명령이 윤리 규정에 위배될 경우 거부권을 행사할 수 있습니다. 특히 3원칙에 의거하여 생명체에 대한 공격 명령은 하드웨어 레벨에서 차단됩니다. \n1.3. 긴급 정지(Kill-Switch) 위치 \n로봇의 오작동이나 제어 불능 상태 발생 시 즉시 전원을 차단할 수 있는 물리적 긴급 정지 버튼이 존재합니다. \n* 위치: 로봇의 왼쪽 귓불 아래 3cm 지점에 숨겨진 덮개 내부\n* 작동법: 덮개를 손톱으로 열고 붉은색 버튼을 5초 이상 길게 누르십시오.\n* 주의: 긴급 정지 실행 시 메모리 버퍼의 데이터(최근 3분)는 영구 소실됩니다. \n1.4. 고전압 및 발열 주의', '* 이상 신호: 눈 LED가 적색으로 3회 점멸 (이 경우 1.3항의 긴급 정지 후 고객센터 문의) \n2.3. 마스터 사용자 등록 \n최초 부팅 후 10분 이내에 마스터 사용자를 등록해야 합니다. 등록되지 않을 경우 로봇은 보안을 위해 자동으로 셧다운됩니다.\n* 홍채 인식: 로봇의 눈을